In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/transformers/天池-入门NLP - 新闻文本分类')

In [ ]:
#安装
!pip install transformers datasets

     |████████████████████████████████| 3.1 MB 2.0 MB/s 
     |████████████████████████████████| 290 kB 72.4 MB/s 
     |████████████████████████████████| 59 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 67.0 MB/s 
     |████████████████████████████████| 895 kB 63.9 MB/s 
     |████████████████████████████████| 3.3 MB 23.6 MB/s 
     |████████████████████████████████| 132 kB 50.1 MB/s 
     |████████████████████████████████| 1.1 MB 61.9 MB/s 
     |████████████████████████████████| 243 kB 57.0 MB/s 
     |████████████████████████████████| 160 kB 67.8 MB/s 
     |████████████████████████████████| 271 kB 50.3 MB/s 
     |████████████████████████████████| 192 kB 70.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#定义全局超参数
batch_size=64
lr=1e-5
dropout=0.15
epoch=1

In [ ]:
#将3750/648/900改成标点符号，删除原text列，新增列重名为text列
import re
def replacepunc(x):
  x=re.sub('3750',",",x)
  x=re.sub('900',".",x)
  x=re.sub('648',"!",x)
  return x

In [ ]:
#保存分词器模型并重新加载
#tokenizer.save("tokenizers.json")

from transformers import PreTrainedTokenizerFast
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizers.json",
   model_max_length=512,mask_token='[MASK]',pad_token='[PAD]',unk_token='[UNK]',
   cls_token='[CLS]',sep_token='[SEP]',padding_side='right',
   return_special_tokens_mask=True)
#PreTrainedTokenizerFast中一定要设置mask_token，pad_token等，不然报错

In [ ]:
#加载数据

from datasets import Dataset
import pandas as pd
train_df=pd.read_csv('./train_set.csv',sep='\t',nrows=100).sample(frac=1)

#将训练数据中三个token换成标点
train_df['texts']=train_df['text'].map(lambda x:replacepunc(x))

#准备将text文本首尾截断，各取255tokens
def slipt2(x):
  ls=x.split(' ')
  le=len(ls)
  if le<511:
    return x
  else:
    return ' '.join(ls[:255]+ls[-255:]) 

train_df['summary']=train_df['texts'].apply(lambda x:slipt2(x))

In [ ]:
"""数据量太大的话考虑block_size
block_size = 128
def group_texts(examples):
  total_length=len(examples[list(examples.keys())[0]])
"""

参考HF的notebook[《custom_datasets》](https://github.com/huggingface/notebooks/blob/master/transformers_doc/custom_datasets.ipynb)

In [ ]:
#最后一个epoch读取全部数据
batch_size=8
#数据预处理
train_encoding=fast_tokenizer(list(train_df['summary']),truncation=True,padding=True,return_tensors='pt')

import torch
# 数据集读取
class XFeiDataset(Dataset):
  def __init__(self,encodings,labels):
    self.encodings=encodings
    self.labels=labels
  
  # 读取单个样本
  def __getitem__(self,idx):
    item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels']=torch.tensor(int(self.labels[idx]))
    return item
  
  def __len__(self):
    return len(self.labels)

train_dataset=XFeiDataset(train_encoding,list(train_df['label']))

# 单个读取到批量读取
from torch.utils.data import Dataset, DataLoader,TensorDataset

train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
for x in train_loader:
    print(x)

{'input_ids': tensor([[   1, 1070, 1576,  ...,    3,    3,    3],
        [   1,  455,  617,  ..., 1059,  562,    2],
        [   1,  551, 1456,  ...,    7,  159,    2],
        [   1,  100,  437,  ...,    3,    3,    3]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([ 3,  3,  2, 11])}
{'input_ids': tensor([[   1,  728, 1350,  ...,  403,  266,    2],
        [   1,  262,  872,  ...,    3,    3,    3],
        [   1, 1089,  343,  ..., 1378, 1139,    2],
        [   1,  210, 1546,  ...,  678,  666,    2]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


In [ ]:
#少量数据读取没问题，全量数据会爆,选用更高配置的colab
from sklearn.model_selection import train_test_split

train_data,val_data,train_label,val_label=train_test_split(
    train_df['summary'].iloc[:], 
    train_df['label'].iloc[:],
    test_size=0.1, 
    stratify=train_df['label'].iloc[:])

#数据预处理
train_encoding=fast_tokenizer(list(train_data),truncation=True,padding=True,return_tensors='pt')
val_encoding=fast_tokenizer(list(val_data),truncation=True,padding=True,return_tensors='pt')

import torch
# 数据集读取
class XFeiDataset(Dataset):
  def __init__(self,encodings,labels):
    self.encodings=encodings
    self.labels=labels
  
  # 读取单个样本
  def __getitem__(self,idx):
    item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels']=torch.tensor(int(self.labels[idx]))
    return item
  
  def __len__(self):
    return len(self.labels)

train_dataset=XFeiDataset(train_encoding,list(train_label))
val_dataset=XFeiDataset(val_encoding,list(val_label))

# 单个读取到批量读取
from torch.utils.data import Dataset, DataLoader,TensorDataset


train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
import torch
#检查其中一个批次：
for batch in train_loader:
    break
{k: v.shape for k, v in batch.items()}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


{'attention_mask': torch.Size([64, 512]),
 'input_ids': torch.Size([64, 512]),
 'labels': torch.Size([64]),
 'token_type_ids': torch.Size([64, 512])}

模型结构参考：[《NER:CLUENER2020/BERT-LSTM-CRF/model.py》](https://github.com/hemingkx/CLUENER2020/blob/main/BERT-LSTM-CRF/model.py)

[《bert_bi-lstm代码解读》](https://blog.csdn.net/qq_41821608/article/details/115784406?ops_request_misc=&request_id=&biz_id=102&utm_term=bert+lstm&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-9-115784406.pc_search_all_es&spm=1018.2226.3001.4187)

In [ ]:
from transformers import BertModel
from torch.nn.utils.rnn import pad_sequence
#初始化bert模型
from transformers import BertConfig
import torch.nn as nn
from torch.nn import LSTM 
from torchcrf import CRF

class Bert_LSTM(nn.Module):
  def __init__(self):
    super(Bert_LSTM,self).__init__()
    self.num_labels=14
    self.dropout=nn.Dropout(dropout)
    self.bert=BertModel.from_pretrained("./bert_model/pre_Bert-8-epoch")#这是之前做mlm任务预训练好的bert模型
    for param in self.bert.parameters():
      param.requires_grad=True
    self.classifier=nn.Linear(1024,14)
    
    self.bilstm=nn.LSTM(
        input_size=512, 
        hidden_size=1024 // 2,  # 1024
        batch_first=True,
        num_layers=2,
        dropout=0.5,  # 0.5
        bidirectional=True)

  def forward(self,batch_seqs,batch_seq_masks,batch_seq_segments):

    output=self.bert(input_ids=batch_seqs,attention_mask=batch_seq_masks,token_type_ids=batch_seq_segments)
    pooler_output=output.pooler_output
    last_hidden_state=output.last_hidden_state
    #只有这种写法不会报错，如果是sequence_output,pooler_output=self.bert(**kwags)这种，sequence_output会报错str没有xxx属性。
    #貌似是bert输出有很多，直接用output.last_hidden_state来调用结果（估计是版本问题，坑），关键是输出要打印出来
    if model.train():
      drop_hidden_state=self.dropout(last_hidden_state)

    #output为输出序列的隐藏层，hn为最后一个时刻的隐藏层，cn为最后一个时刻的隐藏细胞
    #双向lstm,将最后一层最后时刻hn两个方向的结果拼接起来（就是最后一层的双向hidden结果）
    lstm_output,(hn,cn)=self.bilstm(drop_hidden_state)
    hidden_last_out=torch.cat([hn[-2],hn[-1]],dim=-1)
    if model.train():
      hidden_last_out=self.dropout(hidden_last_out)


    # 得到判别值
    logits=self.classifier(hidden_last_out)
    return logits   

In [ ]:
#加载模型
model=Bert_LSTM()
model.load_state_dict(torch.load("best_bert_model_3epoch"))
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

优化器写法参考文档[《NER代码：CLUENER2020/BERT-LSTM-CRF/run.py》](https://github.com/hemingkx/CLUENER2020/blob/main/BERT-LSTM-CRF/run.py)

以及HF教程的[自定义训练循环](https://blog.csdn.net/qq_56591814/article/details/120147114)

In [ ]:
#定义优化器
from transformers import AdamW,get_scheduler

train_steps_per_epoch=len(train_loader)
num_training_steps=train_steps_per_epoch*epoch

#定义各模块参数
bert_parameters=list(model.bert.named_parameters())
lstm_parameters=list(model.bilstm.named_parameters())
classifier_parameters=list(model.classifier.named_parameters())
no_decay=['bias','LayerNorm.weight']


optimizer_grouped_parameters=[
    {'params':[p for n,p in bert_parameters if not any(nd in n for nd in no_decay)],
      'lr':lr,'weight_decay':0.01},
    {'params':[p for n,p in bert_parameters if any(nd in n for nd in no_decay)],
      'lr':lr,'weight_decay':0.0},
    {'params':[p for n,p in lstm_parameters if not any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay':0.01},
    {'params':[p for n,p in lstm_parameters if any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay': 0.0},
    {'params':[p for n,p in classifier_parameters if not any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay':0.01},
    {'params':[p for n,p in classifier_parameters if any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay':0.0}]

optimizer=AdamW(optimizer_grouped_parameters,lr=lr,eps=1e-8)

lr_scheduler=get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

In [ ]:
import torch
for idx,j in enumerate(train_loader):
  print(idx,j)
  break

训练、验证、预测部分参照：[《9102年，再不会用bert就out了》](https://zhuanlan.zhihu.com/p/112655246?utm_source=wechat_session&utm_medium=social&utm_oi=1400823417357139968&utm_campaign=shareopn)

[《10分钟 杀入科大讯飞中文相似度 Top10》](https://mp.weixin.qq.com/s/PDwCO-7TUIZfVkQF8aYHiQ)

In [ ]:
#编写训练和验证循环
import time
import numpy as np
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
#加载进度条
from tqdm.auto import tqdm
num_training_steps=train_steps_per_epoch*epoch

progress_bar=tqdm(range(num_training_steps))

def train_and_eval(epoch):
  best_acc=0.0
  criterion=nn.CrossEntropyLoss()
  for i in range(epoch):
    """训练模型"""
    start=time.time()
    model.train()
    print("***** Running training epoch {} *****".format(i+1))
    train_loss_sum=0.0
    for idx,batch in enumerate(train_loader):
      input_ids=batch['input_ids'].to(device)
      attention_mask=batch['attention_mask'].to(device)
      token_type_ids=batch['token_type_ids'].to(device)
      labels=batch['labels'].to(device)

      #计算输出和loss
      logits=model(input_ids,attention_mask,token_type_ids)
      loss=criterion(logits,labels)
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()  
      progress_bar.update(1)

      train_loss_sum+=loss.item()
      if (idx+1)%(len(train_loader)//5)==0: # 只打印五次结果
        print("Epoch {:03d} | Step {:04d}/{:04d} | Loss {:.4f} | Time {:.4f} | Learning rate = {} \n".format(
                  i+1,idx+1,len(train_loader),train_loss_sum/(idx+1),time.time()-start,optimizer.state_dict()['param_groups'][0]['lr']))
      
      #验证模型
    model.eval()
    total_eval_accuracy=0
    total_eval_loss=0
    for batch in val_loader:
      with torch.no_grad():#只有这一块是不需要求导的
      
        input_ids=batch['input_ids'].to(device)
        attention_mask=batch['attention_mask'].to(device)
        token_type_ids=batch['token_type_ids'].to(device)
        labels=batch['labels'].to(device)
        logits=model(input_ids,attention_mask,token_type_ids)
      
      loss=criterion(logits,labels)#计算loss和准确率
      total_eval_loss+=loss.item()

      logits=logits.clone().detach().cpu().numpy()#detach表示复制且不可求导，原tensor不变，仍可求导
      predictions=np.argmax(logits,axis=-1)
      label_ids=labels.to('cpu').numpy()
      total_eval_accuracy+=accuracy_score(predictions,label_ids)

    avg_val_accuracy=total_eval_accuracy/len(val_loader)
    if avg_val_accuracy>best_acc:
      best_acc=avg_val_accuracy
      torch.save(model.state_dict(),"best_bert_model")
    
    print("val_accuracy:%.4f" % (avg_val_accuracy))
    print("Average val loss: %.4f"%(total_eval_loss/len(val_loader)))
    print("time costed={}s \n".format(round(time.time()-start,5)))
    print("-------------------------------")

  0%|          | 0/3125 [00:00<?, ?it/s]

In [ ]:
train_and_eval(epoch)

***** Running training epoch 1 *****


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


Epoch 001 | Step 0625/3125 | Loss 0.0690 | Time 611.3532 | Learning rate = 8.000000000000001e-06 

Epoch 001 | Step 1250/3125 | Loss 0.0687 | Time 1222.2252 | Learning rate = 6e-06 

Epoch 001 | Step 1875/3125 | Loss 0.0686 | Time 1833.6001 | Learning rate = 4.000000000000001e-06 

Epoch 001 | Step 2500/3125 | Loss 0.0685 | Time 2444.5682 | Learning rate = 2.0000000000000003e-06 

Epoch 001 | Step 3125/3125 | Loss 0.0677 | Time 3055.7369 | Learning rate = 0.0 



1. 第一次batch_size=32,bert_lr=5e-5,其它lr=2.5e-4。训练集loss一直下降到0.03，测试集loss在第二个epoch就开始一直震荡，val_loss=0.2左右，acc在0.9460左右震荡

2. 第二次batch_size=32,bert_lr=5e-5，其它lr=2.5e-4。bert的输出dropout概率由0.1升至0.3，scheduler加了一个epoch的warmup。（加了warmup那不是第二个epoch的lr最高，会不会更容易跳过最低点）
- 观察发现第一个epoch，lr最低到最高时，epoch内5次loss都明显下降(2.54-->1.02)。
- 第二个epoch内loss下降非常慢（0.2623-->0.2381）
- 惊鹊代码bert_lr=5e-5,其它是2e-5。test_batch_size=32，train_batch_size=4。dropout=0.15（bert和lstm两次）,没有warmup，epoch=2
3. 原来一直用的15000的数据，难怪准确率这么低

In [ ]:
#编写predict函数
def predict(model,data_loader):
  model.eval()
  test_pred = []
  for batch in data_loader:
    with torch.no_grad():
      input_ids=batch['input_ids'].to(device)
      attention_mask=batch['attention_mask'].to(device)
      token_type_ids=batch['token_type_ids'].to(device)
      logits=model(input_ids,attention_mask,token_type_ids)

      y_pred=torch.argmax(logits,dim=1).detach().cpu().numpy().tolist()
      test_pred.extend(y_pred)
  return test_pred

In [ ]:
#读取测试集
import pandas as pd
from datasets import load_dataset
test_df=pd.read_csv('./test_a.csv',sep='\t',names=['text','label'])#读取测试集，新增一个空列名为label


#将训练数据中三个token换成标点
test_df['texts']=test_df['text'].map(lambda x:replacepunc(x))
#准备将text文本首尾截断，各取255tokens
def slipt2(x):
  ls=x.split(' ')
  le=len(ls)
  if le<511:
    return x
  else:
    return ' '.join(ls[:255]+ls[-255:])
test_df['summary']=test_df['texts'].apply(lambda x:slipt2(x))
test_df['label']=test_df['label'].fillna(0)#将测试集label全部填充为0,为了对应Dataset的读取格式

In [ ]:
test_df=test_df.drop([0],axis=0)#第一行为原列索引，删除
test_df

,text,label,texts,summary
1,5399 3117 1070 4321 4568 2621 5466 3772 4516 2...,0.0,5399 3117 1070 4321 4568 2621 5466 3772 4516 2...,5399 3117 1070 4321 4568 2621 5466 3772 4516 2...
2,2491 4109 1757 7539 648 3695 3038 4490 23 7019...,0.0,2491 4109 1757 7539 ! 3695 3038 4490 23 7019 3...,2491 4109 1757 7539 ! 3695 3038 4490 23 7019 3...
3,2673 5076 6835 2835 5948 5677 3247 4124 2465 5...,0.0,2673 5076 6835 2835 5948 5677 3247 4124 2465 5...,2673 5076 6835 2835 5948 5677 3247 4124 2465 5...
4,4562 4893 2210 4761 3659 1324 2595 5949 4583 2...,0.0,4562 4893 2210 4761 3659 1324 2595 5949 4583 2...,4562 4893 2210 4761 3659 1324 2595 5949 4583 2...
5,4269 7134 2614 1724 4464 1324 3370 3370 2106 2...,0.0,4269 7134 2614 1724 4464 1324 3370 3370 2106 2...,4269 7134 2614 1724 4464 1324 3370 3370 2106 2...
...,...,...,...,...
49996,3725 4498 2282 1647 6293 4245 4498 3615 1141 2...,0.0,3725 4498 2282 1647 6293 4245 4498 3615 1141 2...,3725 4498 2282 1647 6293 4245 4498 3615 1141 2...
49997,4811 465 3800 1394 3038 2376 2327 5165 3070 57...,0.0,4811 465 3800 1394 3038 2376 2327 5165 3070 57...,4811 465 3800 1394 3038 2376 2327 5165 3070 57...
49998,5338 1952 3117 4109 299 6656 6654 3792 6831 21...,0.0,5338 1952 3117 4109 299 6656 6654 3792 6831 21...,5338 1952 3117 4109 299 6656 6654 3792 6831 21...
49999,893 3469 5775 584 2490 4223 6569 6663 2124 168...,0.0,893 3469 5775 584 2490 4223 6569 6663 2124 168...,893 3469 5775 584 2490 4223 6569 6663 2124 168...


In [ ]:
#加载到datalodar并预处理
# 数据集读取

from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch
class XFeiDataset(Dataset):
  def __init__(self,encodings,labels):
    self.encodings=encodings
    self.labels=labels
  
  # 读取单个样本
  def __getitem__(self,idx):
    item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels']=torch.tensor(int(self.labels[idx]))
    return item
  
  def __len__(self):
    return len(self.labels)
test_encoding=fast_tokenizer(list(test_df['summary']),truncation=True,padding=True,return_tensors='pt')
test_dataset=XFeiDataset(test_encoding,list(test_df['label']))
test_loader=DataLoader(test_dataset,batch_size=64)

In [ ]:
#用trainer预测结果并保存
torch.save(model.state_dict(),"best_lstm_whole_4epoch")
model.load_state_dict(torch.load("best_lstm_whole_4epoch"))
model.to(device)

predictions=predict(model,test_loader)

pd.DataFrame({'label':predictions}).to_csv('bert_lstm_whole_4epoch.csv',index=None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [ ]:
"""
#测试lstm模型输出
model2=nn.LSTM(
        input_size=512, 
        hidden_size=1024 // 2,  # 1024
        batch_first=True,
        num_layers=3,
        dropout=0.1,  # 0.5
        bidirectional=True)
tests=torch.rand(5,3,512)

output,(hn,cn)=model2(tests)
dhn=torch.cat([hn[-2],hn[-1]],dim=-1)#最后一层双向末端结果拼接，大小batch，dim*2

print(output.shape)
print(hn.shape)
print(cn.shape)
#output输出是batch，len，dim*2（双向的话）
#hn和cn都是每个样本一个最后时刻的向量，为1，batch，dim。如果是多层就是num_layers*2，batch，dim
print(dhn.shape)
"""

In [ ]:
"""
import torch
a=torch.rand(3,5,5)
b=torch.rand(3,1,5)
print(a)
print(b)
"""

tensor([[[0.9562, 0.7617, 0.2613, 0.6412, 0.1098],
         [0.5196, 0.6930, 0.0513, 0.7487, 0.5447],
         [0.6831, 0.9386, 0.9693, 0.4979, 0.9142],
         [0.5321, 0.8190, 0.6658, 0.8020, 0.0691],
         [0.3383, 0.3779, 0.2897, 0.1134, 0.2509]],

        [[0.7673, 0.7032, 0.2328, 0.7086, 0.8448],
         [0.3034, 0.7003, 0.3047, 0.2293, 0.8905],
         [0.4470, 0.3945, 0.7205, 0.6552, 0.7908],
         [0.1026, 0.3110, 0.7278, 0.9898, 0.9549],
         [0.4184, 0.2673, 0.2727, 0.7099, 0.8876]],

        [[0.7328, 0.5335, 0.2701, 0.1489, 0.8574],
         [0.2605, 0.9681, 0.9864, 0.8795, 0.3304],
         [0.0284, 0.7669, 0.3259, 0.7404, 0.7547],
         [0.1082, 0.5845, 0.8316, 0.4850, 0.4264],
         [0.3979, 0.6959, 0.1624, 0.4869, 0.2568]]])
tensor([[[0.3852, 0.3308, 0.1632, 0.3874, 0.7840]],

        [[0.3321, 0.7246, 0.7355, 0.5728, 0.8681]],

        [[0.4072, 0.4212, 0.6390, 0.5725, 0.8822]]])
